In [145]:
# KEYWORDS: MinHash(LSH) 

In [171]:
import numpy as np
import pandas as pd 
import string
import random


## Reading Data Files 

In [147]:
# The target file includes the original sequences that encodes the data. 
with open('targetBlawat et al.LDPC.txt') as f:
    reads = f.readlines()
for i in range(0,len(reads)):
    reads[i] = reads[i][0:len(reads[i])-1]
reads 

['...',
 ...]

In [148]:
np.size(reads)

9608

In [152]:
# The errors_shuffled file include the erroneous copies shuffled. 
with open('errors_shuffled.txt') as f:
    reads_err = f.readlines()
for i in range(0,len(reads_err)):
    reads_err[i] = reads_err[i][0:len(reads_err[i])-1]
# reads_err = reads_err[0:1000] 

In [153]:
np.size(reads_err)

1196656

In [154]:
# evyat.txt include the erroneous copies clustered and labeled. 
with open('evyat.txt') as f:
    reads_cl = f.readlines()
for i in range(0,len(reads_cl)):
    reads_cl[i] = reads_cl[i][0:len(reads_cl[i])-1]
reads_cl

['TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 '*****************************',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGCGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGAAACCCACACTGGGTTGTACTTGGTCAAA',
 'TGAATAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTTGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAACACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAAAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAGACGAGTCCCGGAAACCCACACT

## Functions


In [173]:
def hash_fun_ind(x,a,w,l): 
    cnt = 0 
    for i in range(0,len(x)):
        if x[i] == a[cnt]:
            cnt += 1  
        else:
            cnt = 0 
        if cnt == w:
            return i-w+1 
    return -1

def hash_fun(x,a,w,l):
    ind = hash_fun_ind(x,a,w,l)
    return x[ind:min(len(x),ind+w+l)]

def ind_st(st):
    #All 3-grams: AAA,AAG,AAC,AAT,AGA,AGC,...,TTA,TTG,TTC,TTT
    # A-0, G-1, C-2, T-3
    # index of CAT = Decimal representaion of (203)
    N_q = {"A":0,"C":1,"G":2,"T":3}
    dec = 0
    for i in range(0,len(st)):
        dec += N_q[st[i]]*(4**(len(st)-i-1))
    return dec 
    
def bin_sig(x,q):
    bs = [0]*(4**q) 
    for i in range(0,len(x)-q+1):
        st = x[i:i+q]
        bs[ind_st(st)] = 1
    return bs 
        
def ham_dis(x,y):
    dis = 0
    for i in range(0,len(x)):
        if(x[i]!= y[i]):
            dis += 1
    return dis 

def rand_perm(w):
    return ''.join(random.choice('ACGT') for _ in range(w))

def cluster(x,y,parent):
    parent[x]= min(parent[x],parent[y])
    parent[y]= min(parent[x],parent[y])

In [187]:
rand_perm(4)

'ATAA'

In [156]:
hash_fun("abc","c",1,2)

'c'

In [157]:
time hash_fun(reads_err[0],"GA",2,2)

CPU times: user 99 µs, sys: 19 µs, total: 118 µs
Wall time: 59.8 µs


'GAGA'

In [158]:
time bin_sig("TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA",2)

CPU times: user 86 µs, sys: 0 ns, total: 86 µs
Wall time: 89.2 µs


[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]

In [159]:
ham_dis("ACT","ATG")

2

In [160]:
q =  4
x = "GGGAGACTCGCAATATAGAATACATACTTGACGTAGCAGTCTTTTATGGA"
y = "TCATCTCTAAGCGGCGAGCTACAATCCGCTCCGAAGCGCTCCCTAGTAAC"
ham_dis(bin_sig(x,q),bin_sig(y,q))

82

In [161]:
# edit_dis(x,y)
len(reads_err)

1196656

## A Local Computaion Step of the Algorithm 

In [162]:
# GLOBAL CLUTERING DENOTED BY C_til
# Parent stores cluster id corresponding to each read
# Disjoint set Union Problem 

C_til  = [0]*(len(reads_err))
parent = [0]*(len(reads_err))
for i in range(0,len(reads_err)):
    C_til[i] = (i,reads_err[i]) 
    parent[i] = i
C_til

[(0, 'GGGAGACTCGCAATATAGAATACATACTTGACGTAGCAGTCTTTTATGGA'),
 (1, 'GCAGAGGATTCCTGGGCCGCTGATCCAGTTAACACATGTGATGGCCACAC'),
 (2, 'ATCATGGTCAAGAGGTTGATTTCGATCAACTGAGCATCCCGTCTGCGGTTG'),
 (3, 'TCATCTCTAAGCGGCGAGCTACAATCCGCTCCGAAGCGCTCCCTAGTAAC'),
 (4, 'ATCGGTACCCTCAAACTAGCATGGAACTTCTATCGGATCGCTGGCAGATA'),
 (5, 'GAGTGGTCTCGAAGACATGCAACCCAGCACATACAACACCCGAGCGGTAAT'),
 (6, 'TCGCTGTTAAATCGTCACACCACGTCAACTATACGTAGTACCTCCCTGAC'),
 (7, 'AGGCCGCTTATAGAGGACGTCTGTTCAGGTCATCGGTATGTCTCGTGAG'),
 (8, 'GTATCCTACTCAGCTTAAGGGGCACACGCCAGCGCATGCACACACTGCTCC'),
 (9, 'CTATCTTCACAAATACCTGTTCCTGCTCGCCTAGTTCTGCCTGGACGACT'),
 (10, 'CTACCCAGTAGCGCATACGTTAGCTATTATGAGGCTTGATGGTCGAAGTGG'),
 (11, 'TAGGGTTGCATCTTTGCATAATAAACTTGGCCATAGCGTGTCTCCCTCCC'),
 (12, 'GTTGTCAAGGGACATGGATTTGCCCCTCGGGATGTATTCAGGGATTCA'),
 (13, 'CGCATTGGGTAGGACCAACATCTGCTCCGCATGTGAGGGTTGATATGATG'),
 (14, 'GGTTATGGCTACTTGTCTACGGTGACTAGTTGACCGCTTAGCCTCCCGGT'),
 (15, 'CCATGTGAGGAGACTGGCTACCTCGGCAGGTAAGTTCGGGCATAACCGATT'),
 (16, 'ACTTGAAACGTAGACGCACGATATC

In [163]:
# Step 1: Sample a Hash Function 
# Step 2: Compute Hash of a random representative of each cluster 
# Step 3: Merge two clusters with same hash if ham_dis(bin_sig(x),bin_sig(y)) <= th_high
a = "GGAA"
w = len(a)
l = int(w/2)
hash_C_til = [0]*(len(reads_err))

for i in range(0,len(C_til)):
    hash_C_til[i] = (i,hash_fun(C_til[i][1],a,w,l))

In [164]:
hash_C_til

[(0, ''),
 (1, ''),
 (2, ''),
 (3, ''),
 (4, 'GGAACT'),
 (5, ''),
 (6, ''),
 (7, ''),
 (8, ''),
 (9, ''),
 (10, ''),
 (11, ''),
 (12, ''),
 (13, ''),
 (14, ''),
 (15, ''),
 (16, ''),
 (17, ''),
 (18, ''),
 (19, 'GGAAGA'),
 (20, ''),
 (21, ''),
 (22, ''),
 (23, ''),
 (24, ''),
 (25, ''),
 (26, ''),
 (27, ''),
 (28, ''),
 (29, 'GGAATC'),
 (30, 'GGAACT'),
 (31, ''),
 (32, 'GGAAGT'),
 (33, ''),
 (34, ''),
 (35, ''),
 (36, ''),
 (37, ''),
 (38, ''),
 (39, 'GGAAAC'),
 (40, ''),
 (41, 'GGAATA'),
 (42, ''),
 (43, ''),
 (44, ''),
 (45, ''),
 (46, ''),
 (47, ''),
 (48, 'GGAATT'),
 (49, ''),
 (50, ''),
 (51, ''),
 (52, ''),
 (53, ''),
 (54, ''),
 (55, ''),
 (56, ''),
 (57, ''),
 (58, ''),
 (59, ''),
 (60, ''),
 (61, 'GGAAGA'),
 (62, ''),
 (63, ''),
 (64, ''),
 (65, ''),
 (66, 'GGAAAG'),
 (67, 'GGAATT'),
 (68, 'GGAACA'),
 (69, ''),
 (70, ''),
 (71, ''),
 (72, ''),
 (73, 'GGAAAA'),
 (74, ''),
 (75, ''),
 (76, ''),
 (77, ''),
 (78, 'GGAAAC'),
 (79, ''),
 (80, ''),
 (81, ''),
 (82, ''),
 (83, ''),
 (

In [165]:
hash_C_til.sort(key=lambda x:x[1])
hash_C_til

[(0, ''),
 (1, ''),
 (2, ''),
 (3, ''),
 (5, ''),
 (6, ''),
 (7, ''),
 (8, ''),
 (9, ''),
 (10, ''),
 (11, ''),
 (12, ''),
 (13, ''),
 (14, ''),
 (15, ''),
 (16, ''),
 (17, ''),
 (18, ''),
 (20, ''),
 (21, ''),
 (22, ''),
 (23, ''),
 (24, ''),
 (25, ''),
 (26, ''),
 (27, ''),
 (28, ''),
 (31, ''),
 (33, ''),
 (34, ''),
 (35, ''),
 (36, ''),
 (37, ''),
 (38, ''),
 (40, ''),
 (42, ''),
 (43, ''),
 (44, ''),
 (45, ''),
 (46, ''),
 (47, ''),
 (49, ''),
 (50, ''),
 (51, ''),
 (52, ''),
 (53, ''),
 (54, ''),
 (55, ''),
 (56, ''),
 (57, ''),
 (58, ''),
 (59, ''),
 (60, ''),
 (62, ''),
 (63, ''),
 (64, ''),
 (65, ''),
 (69, ''),
 (70, ''),
 (71, ''),
 (72, ''),
 (74, ''),
 (75, ''),
 (76, ''),
 (77, ''),
 (79, ''),
 (80, ''),
 (81, ''),
 (82, ''),
 (83, ''),
 (84, ''),
 (85, ''),
 (86, ''),
 (87, ''),
 (88, ''),
 (89, ''),
 (90, ''),
 (91, ''),
 (92, ''),
 (93, ''),
 (94, ''),
 (95, ''),
 (96, ''),
 (97, ''),
 (98, ''),
 (99, ''),
 (100, ''),
 (101, ''),
 (102, ''),
 (104, ''),
 (105, ''),
 (1

In [166]:
th_high = 75
cnt =0
for i in range(0,len(reads_err)-1):
    if(hash_C_til[i][1] == ""):
        continue 
    else:
        if(hash_C_til[i][1] == hash_C_til[i+1][1]):
            x = reads_err[hash_C_til[i][0]]
            y = reads_err[hash_C_til[i+1][0]]
            if(ham_dis(bin_sig(x,q),bin_sig(y,q)) <= th_high):
                cnt +=  1
                cluster(hash_C_til[i][0],hash_C_til[i+1][0],parent)
cnt

156046

In [167]:
parent

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 4,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 19,
 62,
 63,
 64,
 65,
 66,
 48,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 39,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 19,
 104,
 105,
 106,
 107,
 108,
 66,
 110,
 111,
 112,
 113,
 114,
 115,
 73,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 41,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 48,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 39,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 41,
 156,
 157,
 158,
 159,
 160,
 140,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 140,
 29,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 66,
 185,
 29,

# Clustering Algorithm

In [ ]:
with open('errors_shuffled.txt') as f:
    reads_err = f.readlines()
for i in range(0,len(reads_err)):
    reads_err[i] = reads_err[i][0:len(reads_err[i])-1]
    
C_til  = [0]*(len(reads_err))
parent = [0]*(len(reads_err))
for i in range(0,len(reads_err)):
    C_til[i] = (i,reads_err[i]) 
    parent[i] = i

local_comm = 10

for i in range(0,local_comm):
    w = 4 
    a = rand_perm(w)
    l = 4
    hash_C_til = [0]*(len(reads_err))

    for i in range(0,len(C_til)):
        hash_C_til[i] = (i,hash_fun(C_til[i][1],a,w,l))
    
    hash_C_til.sort(key=lambda x:x[1])
    th_high = 75
    cnt =0
    for i in range(0,len(reads_err)-1):
        if(hash_C_til[i][1] == ""):
            continue 
        else:
            if(hash_C_til[i][1] == hash_C_til[i+1][1]):
                x = reads_err[hash_C_til[i][0]]
                y = reads_err[hash_C_til[i+1][0]]
                if(ham_dis(bin_sig(x,q),bin_sig(y,q)) <= th_high):
                    cnt +=  1
                    cluster(hash_C_til[i][0],hash_C_til[i+1][0],parent)
                    
    print(i,cnt)

1196654 169830
1196654 169830


In [193]:
parent

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 164,
 168,
 169,
 170,
 82,
 96,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 2,
 182,
 149,
 184,
 185

In [ ]:
def parent_to_cluster(parent):
    for i in range(0,len(parent)):
        if(parent[i]==i):
            